<a href="https://colab.research.google.com/github/kaniarasann/Langchain/blob/main/07.Tools_with_Orchestration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [584]:
#!pip install langchain langgraph langchain_groq pydantic feedparser beautifulsoup4 langchain-community langchain-exa

Imports

In [585]:
from pydantic import BaseModel,Field
from datetime import datetime
from dateutil import parser as dtparser
import feedparser
from langchain_groq import ChatGroq
from google.colab import userdata
from langchain_core.prompts import PromptTemplate
from typing import List
import operator
from typing_extensions import Annotated
from langchain_exa import ExaSearchResults
from langgraph.constants import Send
from langgraph.graph import StateGraph, START, END
import re

Get top 5 news from the economic times this can be achieved with the help of RSS Feed : https://economictimes.indiatimes.com/markets/rssfeeds/1977021501.cms

In [586]:
url = 'https://economictimes.indiatimes.com/markets/rssfeeds/1977021501.cms'

In [587]:
# Class required for converting the Feed to pydantic class
class EconomicTime(BaseModel):
    title: str =''
    link: str = ''
    published: datetime =''
    content :str =''

In [588]:
#feed parse the Url add the nessecary package
feeds = feedparser.parse(url)

<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()


In [589]:
#iterate through the first 5 feed
economic_times:list[EconomicTime] = []
for feed in feeds.entries[:5]:
    economic_times.append(EconomicTime(
        title = feed.title,
        link = feed.link,
        published = dtparser.parse(feed.published)

    ))


In [590]:
#get the content by using beautiful soup
from bs4 import BeautifulSoup
import requests

for news in economic_times:
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(news.link, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    article_div = soup.find("article")
    article_text = article_div.get_text(strip=True)
    news.content = article_text

In [591]:
#create LLM
groq_key = userdata.get('groq_key')
llm = ChatGroq(api_key=groq_key, model="qwen/qwen3-32b",max_retries=5)
llm_reasoning = ChatGroq(api_key=groq_key, model="meta-llama/llama-4-scout-17b-16e-instruct",max_retries=5)
llm_deep_seek = ChatGroq(api_key=groq_key, model="deepseek-r1-distill-llama-70b",max_retries=5)
search_tool = ExaSearchResults(exa_api_key= userdata.get('exa_api_key'))


In [592]:
class Stock(BaseModel):
    name: str = Field(description="Name of the stock")
    news_abstract: str = Field(description="Abstract of the news article related to the stock")

In [593]:
llm_stock_news = llm.with_structured_output(Stock)

In [594]:
stock_news:list[Stock] = []
def abstract_stock_new(news):
  prompt_stock = PromptTemplate.from_template("Following is the news:{news}. provide the company name and abstract news ").format(news=news.content)
  return llm_stock_news.invoke(prompt_stock)

for news in economic_times[:1]:
  stock_news.append(abstract_stock_new(news))


In [595]:
#Create State
class State(BaseModel):
    company_name: str = Field(description="Name of the company")
    news_abstract: str = Field(description="Abstract of the news article related to the company")
    queries:List[str] = []
    completed_queries: Annotated[List[str], operator.add] = []
    final_result: str = ''

class Query(BaseModel):
   query:str = ''

class Queries(BaseModel):
  queries:List[str] = []

In [596]:
#Worker State
class WorkerState(BaseModel):
    queries: str = ''
    completed_queries: Annotated[List[str], operator.add] = []

In [597]:
llm_queries = llm_reasoning.with_structured_output(Queries)
def orchestrator(state:State):
  promot_template = PromptTemplate.from_template("Your are Stock Market expert. Based on the following news list out the impacts in short / long:{news}").format(news=state.news_abstract)
  res = llm_queries.invoke(promot_template)
  return {'queries':res.queries}

def llm_call(query):
    """Worker queries web for queries"""
    search_results = search_tool._run(
          query=query['query'],
          num_results=1,
          text_contents_options=True,
          highlights=True,
      )
    prompt_stock = PromptTemplate.from_template("Following is the news:{news}. provide the abstract").format(news=search_results)
    res_abs = llm.invoke(prompt_stock)
    return {'completed_queries':[remove_think_blocks(res_abs.content)]} # Wrap the content in a list

def assign_workers(state: State):
    return [Send("llm_call", {"query": s}) for s in state.queries]

def synthesizer(state: State):
    body = "".join(state.completed_queries)
    promot_template = PromptTemplate.from_template("Your are Stock Market expert.Provide an abstract on the following news and do a analysis:{news}").format(news=body)
    res = llm_deep_seek.invoke(promot_template)
    return {"final_result": remove_think_blocks(res.content)}

def remove_think_blocks(text: str) -> str:
    # This pattern removes everything between <think> and </think>, including the tags
    return re.sub(r"<think>.*?</think>", "", text, flags=re.DOTALL).strip()


In [598]:
graph = StateGraph(State)
graph.add_node("orchestration", orchestrator)
graph.add_node("llm_call", llm_call)
graph.add_node("synthesizer", synthesizer)

graph.add_edge(START, "orchestration")
graph.add_conditional_edges("orchestration", assign_workers, ["llm_call"])
graph.add_edge("llm_call", "synthesizer")
graph.add_edge("synthesizer", END)

builder = graph.compile()

In [599]:
#state = State(company_name="TCS",news_abstract=)
selector = stock_news[0]
state = State(company_name=selector.name,news_abstract=selector.news_abstract)

In [600]:
res_date = builder.invoke(state)


RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `qwen/qwen3-32b` in organization `org_01jzb4yhhpe01ampb9qw2edk5s` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Used 9360, Requested 3498. Please try again in 1m8.584999999s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

In [ ]:
res_date